# Introduction
This notebook was the coding part of my bachelor thesis, titled **The Effect of Fluorination on the Structure and Dynamics of the Hexapeptide VG(Abu)APG**. In this project, I tried to isolate the effect of fluorinated Abu (mono-, di- and trifluorination) by studying it on a small hexapeptide, which is derived from VGLAPG to study the structure and its kinetics.

The simulation itself was carried on GROMACS 2019.1 using the topology created in PyMol. Here we only analyze their structure by analyzing their Ramachandran plot, side chain torsion angle profiles ($\chi_1$), end-to-end distance and the formed intramolecular hydrogen bonds.

In this published project, I only use a shorter trajectory length (100 ns instead of 500 ns) due to the size of the file. In the production run in GROMACS, each hexapeptides were simulated for a total time of 100 ns with 2 fs timestep and the trajectories were saved every picosecond, so in the end we have 100.000 data points.

A visualization of the hexapeptide:

![VGXAPG%20uncapped.png](./VGXAPG_uncapped.png)

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import mdtraj as md
import seaborn as sns
import pandas as pd
%matplotlib inline
from matplotlib.cm import ScalarMappable
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
import warnings
warnings.filterwarnings("ignore")

# publication-ready graph settings
plt.rcParams['figure.figsize'] = [7, 5]
plt.rc('font', family='serif')
plt.rc('xtick', labelsize='medium')
plt.rc('ytick', labelsize='medium')

In [ ]:
fluorinations = {
  "abu": { "name": "Abu" },
  "mfe": { "name": "Mfe" },
  "dfe": { "name": "Dfe" },
  "tfe": { "name": "Tfe" },
}

# Loading Trajectories

After generating the topology file in PyMol, a series of 100 ns trajectory from 4 different hexapeptide mutants were generated in Gromacs 2019.1. The trajectories are then being loaded using `mdtraj` package.

In [ ]:
for k, f in fluorinations.items():
  f["trajectory"] = md.load_xtc("traj_%s_100.xtc" % (k), top = "md_%s.pdb" % (k))

## Sliced trajectory

to build its mean, we slice the trajectory into 5 small portions with the same length of 20.000 ns and build the average over them.

In [ ]:
slice = 20_000

for k, f in fluorinations.items():
  f["sliced_trajectory"] = [f["trajectory"][slice*x:slice*(1+x)] for x in range(5)]

# Fitted Trajectory

In [ ]:
def calculate_fitted_trajectory(x, f):
  return f["sliced_trajectory"][x].superpose(
      f["sliced_trajectory"][x],
      frame = 0,
      atom_indices = f["sliced_trajectory"][x].topology.select("protein and backbone"),
      ref_atom_indices = None,
      parallel = True
    ).center_coordinates()

for k, f in fluorinations.items():
  f["fitted_trajectory"] = [calculate_fitted_trajectory(x, f) for x in range(5)]

# Sidechain and Backbone

In [ ]:
for k, f in fluorinations.items():
    f["sidechain"] = [x.topology.select("sidechain") for x in f["sliced_trajectory"]]
    f["backbone"] = [x.topology.select("backbone") for x in f["sliced_trajectory"]]

## Root-mean-square deviation of atomic positions (RMSD)

Root-mean-square deviation of atomic positions (RMSD) is a measure of the average distance between the atoms (usually the backbone atoms) of superimposed proteins.

In [ ]:
for k, f in fluorinations.items():
  f["rmsd"] = md.rmsd(f["trajectory"], f["trajectory"], frame=0, atom_indices=None)

In [ ]:
fig, ax = plt.subplots(1,4, figsize=(12,3), sharex=True, sharey=True)

for i, (k, f) in enumerate(fluorinations.items()):
  sns.kdeplot(f["rmsd"], ax = ax[i])
  ax[i].set_title(f["name"])

fig.text(0.4, 1, "RMSD of total length of Abu, Mfe, Dfe, and Tfe", fontsize=14)
fig.text(0.5, 0, "RMSD")

plt.show()


# Ramachandran
## Of the total length 100 ns
still using unsliced trajectory

In [ ]:
amino_acids = { x:{} for x in ["val", "gly", "exg", "ala", "pro"] }

In [ ]:
for k, f in fluorinations.items():
  f["phi"] = md.compute_phi(f["trajectory"], periodic=True)
  f["psi"] = md.compute_psi(f["trajectory"], periodic=True)

# Coordinate of phi and psi

Taking abu as the input because in all 4 they are the same

In [ ]:
r_phi = fluorinations["abu"]["phi"][0]
r_psi = fluorinations["abu"]["psi"][0]

# Phi and psi for each amino acid

In [ ]:
for i, (k, acid) in enumerate(amino_acids.items()):
  acid["phi"] = r_phi[i]
  acid["psi"] = r_psi[i]

# Rad2deg

In [ ]:
for k, f in fluorinations.items():
  f["angles"] = { k_acid: np.rad2deg(md.compute_dihedrals(f["trajectory"], (acid["phi"], acid["psi"]))) for k_acid, acid in amino_acids.items() }
  f["hist"] = {}
  f["x"] = {}
  f["hist"]["phi"], f["x"]["phi"] = np.histogram(f["angles"]["pro"], bins=12, density=True)
  f["hist"]["psi"], f["x"]["psi"] = np.histogram(f["angles"]["pro"], bins=12, density=True)

In [ ]:
# This code is weird because 5 angles:
# angles_mfe_val, angles_mfe_gly, angles_mfe_exg, angles_mfe_ala, angles_mfe_pro
# mapped into 1 hist_mfe_phi

# for i in name:
#     for j in aa:
#         globals()['hist_%s_phi' % i], globals()['x_%s_phi' % i] = np.histogram(globals()['angles_%s_%s' % (i,j)], bins = 12, density = True)
#         globals()['hist_%s_psi' % i], globals()['x_%s_psi' % i] = np.histogram(globals()['angles_%s_%s' % (i,j)], bins = 12, density = True)

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(11, 11))
ticks = [*range(-180, 181, 60)]
plt.setp(ax, xticks=ticks, yticks=ticks, xlim=(-180, 180), ylim=(-180, 180))

for i, (k, f) in enumerate(fluorinations.items()):
  ax[i].set_box_aspect(1)
  ax[i].axhline(y=0, color='k', lw=0.5)
  ax[i].axvline(x=0, color='k', lw=0.5)
  
  ax[i].hist2d(f["angles"]["exg"][:, 0], f["angles"]["exg"][:, 1], bins=(100, 100), cmin=1)
  ax[i].set_title(f["name"])
  ax[i].grid(b=True, color="black", alpha=0.2)

cax = fig.add_axes([0.055, 0.3, 0.93, 0.03])
cmap = plt.get_cmap("viridis")
norm = mpl.colors.Normalize(vmin=fluorinations["tfe"]["hist"]["psi"].min(), vmax=fluorinations["tfe"]["hist"]["psi"].max())
sm = ScalarMappable(norm = norm, cmap=cmap)
sm.set_array([])
cbar = fig.colorbar(sm, cax=cax, orientation='horizontal')

fig.text(0.5, 0.65, "Ramachandran Probability Density", ha='center', fontsize=14)
fig.text(0.5, 0.35, "phi angle $\phi$", ha='center', fontsize = 12)
fig.text(-0.01, 0.5, 'psi angle $\psi$', va='center', rotation='vertical', fontsize = 12)
fig.text(0.5, 0.25, "Probability Density", ha='center', fontsize=12)

plt.tight_layout()
plt.show()

In the figure above, it is shown that for each hexapeptide, they do exist as right-handed $\alpha$-Helix and $\beta$ sheets. There is a trend that the more fluorinated a hexapeptide is, the less area they visit in the Ramachandran plot. This can be interpreted as lesser flexibility in the hexapeptides, especially for TfeGly.

In this shorter simulation, none of them visit the left-handed $\beta$ sheets. This may indicate either they do not visit the $\beta$ sheets region at all, or we need a longer simulation length.

# side chain torsion angle profiles $\chi_1$

In [ ]:
for k, f in fluorinations.items():
  f["chi1"] = md.compute_chi1(f["trajectory"], periodic=True)

c_chi1 = fluorinations["abu"]["chi1"][0]

# AA WITH CHI1 IN THE CHAIN: VAL, EXG, PRO

In [ ]:
chi1_chains = ["val", "exg", "pro"]

for i, chain in enumerate(chi1_chains):
    amino_acids[chain]["angle"] = c_chi1[i]

# for i in range(len(chi1_chain)):
#     globals()['angle_%s' % chi1_chain[i]] = np.array(c_chi1)[:,i]

### for whole series

In [ ]:
for i in name:
    for j in chi1_chain:
        globals()['dihedral_%s_%s' % (i, j)] = np.rad2deg(
            md.compute_dihedrals(globals()['traj_%s' % i], globals()['angle_%s' % j]))

In [ ]:
for k, f in flourinations.items():
  f["dihedral"] = [ np.rad2deg(md.compute_dihedrals(f["trajectory"], amino_acids[chain]["angle"])) for chain in chi1_chains ]

In [ ]:
dihedral_abu_exg

### for sliced with length = 20000 ns each

In [ ]:
for i in name:
    for j in chi1_chain:
        for k in range(1,6):
            globals()['dihedral_%s_%s_%s' % (i, j, k)] = np.rad2deg(
                md.compute_dihedrals(globals()['%s_%s' % (i,k)], globals()['angle_%s' % j]))

In [ ]:
dihedral_abu_exg_1

### visualization

In [ ]:
bins_ = 100

for i in name:
    for j in chi1_chain:
        globals()['h_chi1_%s_%s' % (i, j)], globals()['e_chi1_%s_%s' % (i, j)] = np.histogram(
            globals()['dihedral_%s_%s' % (i,j)][:,0], bins = bins_, density = True)

In [ ]:
e_chi1 = [globals()['e_chi1_%s_%s' % (i,j)] for i in name for j in chi1_chain]

In [ ]:
x_chi1_abu_val = (e_chi1_abu_val[1:] + e_chi1_abu_val[:-1]) / 2
x_chi1_mfe_val = (e_chi1_mfe_val[1:] + e_chi1_mfe_val[:-1]) / 2
x_chi1_dfe_val = (e_chi1_dfe_val[1:] + e_chi1_dfe_val[:-1]) / 2
x_chi1_tfe_val = (e_chi1_tfe_val[1:] + e_chi1_tfe_val[:-1]) / 2

x_chi1_abu_exg = (e_chi1_abu_exg[1:] + e_chi1_abu_exg[:-1]) / 2
x_chi1_mfe_exg = (e_chi1_mfe_exg[1:] + e_chi1_mfe_exg[:-1]) / 2
x_chi1_dfe_exg = (e_chi1_dfe_exg[1:] + e_chi1_dfe_exg[:-1]) / 2
x_chi1_tfe_exg = (e_chi1_tfe_exg[1:] + e_chi1_tfe_exg[:-1]) / 2

x_chi1_abu_pro = (e_chi1_abu_pro[1:] + e_chi1_abu_pro[:-1]) / 2
x_chi1_mfe_pro = (e_chi1_mfe_pro[1:] + e_chi1_mfe_pro[:-1]) / 2
x_chi1_dfe_pro = (e_chi1_dfe_pro[1:] + e_chi1_dfe_pro[:-1]) / 2
x_chi1_tfe_pro = (e_chi1_tfe_pro[1:] + e_chi1_tfe_pro[:-1]) / 2

In [ ]:
len(x_chi1_abu_pro)

In [ ]:
fig, axs = plt.subplots(3,4, sharex = True, sharey = True)
fig.suptitle("dihedral angle distribution of mutants from VGLAPG", fontsize = "x-large")

#abu
axs[0,0].set_title("Abu")
axs[0,0].plot(x_chi1_abu_val, h_chi1_abu_val)
axs[1,0].plot(x_chi1_abu_exg, h_chi1_abu_exg)
axs[2,0].plot(x_chi1_abu_pro, h_chi1_abu_pro)

#mfe
axs[0,1].set_title("mfe")
axs[0,1].plot(x_chi1_mfe_val, h_chi1_mfe_val)
axs[1,1].plot(x_chi1_mfe_exg, h_chi1_mfe_exg)
axs[2,1].plot(x_chi1_mfe_pro, h_chi1_mfe_pro)

#dfe
axs[0,2].set_title("dfe")
axs[0,2].plot(x_chi1_dfe_val, h_chi1_dfe_val)
axs[1,2].plot(x_chi1_dfe_exg, h_chi1_dfe_exg)
axs[2,2].plot(x_chi1_dfe_pro, h_chi1_dfe_pro)

#tfe
axs[0,3].set_title("tfe")
axs[0,3].plot(x_chi1_tfe_val, h_chi1_tfe_val)
axs[1,3].plot(x_chi1_tfe_exg, h_chi1_tfe_exg)
axs[2,3].plot(x_chi1_tfe_pro, h_chi1_tfe_pro)
#general formatting
for ax in axs.flat:
    ax.label_outer()
    fig.text(0.5, 0, "$\chi_1$ angle", ha = "center", fontsize = 12)
    fig.text(0, 0.5, "count", va = "center", rotation ="vertical", fontsize = 12)
    fig.text(1, 0.25, "pro", va = "center", rotation ="vertical")
    fig.text(1, 0.5, "exg", va = "center", rotation ="vertical")
    fig.text(1, 0.8, "val", va = "center", rotation ="vertical")
    ax.tick_params(axis="x", direction="in")
    ax.set(**{
        "xticks": (-90, 0, 90),
        "xlim": (-180, 180),
        "ylim": (0)
    })


plt.tight_layout()
plt.show()
#fig.savefig("dihedral angle distribution-np.svg")

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, sharex = True, sharey = True, figsize = (10,10))
fig.suptitle('dihedral angle distribution of mutants from VGXAPG',
             fontsize = 16,
            y = .65)

#abu
ax1.set_title("Abu")
ax1.plot(x_chi1_abu_exg, h_chi1_abu_exg)
ax1.set_ylabel(" ")

#mfe
ax2.set_title("MfeGLy")
ax2.plot(x_chi1_mfe_exg, h_chi1_mfe_exg)

#dfe
ax3.set_title("DfeGly")
ax3.plot(x_chi1_dfe_exg, h_chi1_dfe_exg)

#tfe
ax4.set_title("TfeGly")
ax4.plot(x_chi1_tfe_exg, h_chi1_tfe_exg)

#general formatting
ax1.set_box_aspect(1)
ax2.set_box_aspect(1)
ax3.set_box_aspect(1)
ax4.set_box_aspect(1)

fig.text(0.5, 0.32, "$\chi_1$ Angle", ha='center', fontsize = 12)
fig.text(0, 0.5, 'Probability Density', va='center', rotation='vertical', fontsize = 12)

plt.ylim(0, 0.025)
plt.xlim(-180,180)
plt.xticks([-120, -60, 0, 60, 120])
plt.tight_layout()
plt.show()
#fig.savefig("graph/500_chi1.svg", bbox_inches='tight')

### chi1 mean & error

In [ ]:
chi1_chain

In [ ]:
dihedral_abu_val_2

In [ ]:
#### ------------------------------- ####
#### HISTOGRAM FOR SLICED TRAJECTORY ####
#### ------------------------------- ####
for i in name:
    for j in chi1_chain:
        for k in range(1,6):
            globals()['h_chi1_%s_%s_%s' % (i, j, k)], globals()['e_chi1_%s_%s_%s' % (i,j,k)] = np.histogram(globals()['dihedral_%s_%s_%s' % (i,j,k)][:,0], bins = bins_, density = True)

In [ ]:
#### ------------------------------- ####
#### AXIS FOR SLICED TRAJECTORY      ####
#### ------------------------------- ####

for i in name:
    for j in chi1_chain:
        for k in range(1,6):
            globals()['x_chi1_%s_%s_%s' % (i, j, k)] = (globals()['e_chi1_%s_%s_%s' % (i,j,k)][1:] + globals()['e_chi1_%s_%s_%s' % (i,j,k)][:-1]) /2
            
            

In [ ]:
len(x_chi1_abu_val_1)

In [ ]:
#### ------------------------------- ####
#### MEAN FOR SLICED TRAJECTORY      ####
#### ------------------------------- ####

In [ ]:
#### ------------------------------- ####
#### ARRAY FOR SLICED TRAJECTORY     ####
#### ------------------------------- ####

for i in name:
    for j in chi1_chain:
        globals()['array_%s_%s' % (i,j)] = []

for i in name:
    for j in chi1_chain:
        for k in range(1,6):
            globals()['array_%s_%s' % (i,j)].append(np.array(globals()['h_chi1_%s_%s_%s' % (i,j,k)]))

In [ ]:
#### ------------------------------- ####
#### MEAN CHI1                       ####
#### ------------------------------- ####

for i in name:
    for j in chi1_chain:
        globals()['mean_chi1_%s_%s' % (i,j)] = np.mean(globals()['array_%s_%s' % (i,j)], axis = 0)
    
#### ------------------------------- ####
#### ERROR CHI1                      ####
#### ------------------------------- ####

for i in name:
    for j in chi1_chain:
        globals()['error_chi1_%s_%s' % (i,j)] = np.std(globals()['array_%s_%s' % (i,j)], axis = 0)

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, sharex = True, sharey = True, figsize = (10,10))
fig.suptitle("Dihedral Angle $\chi_1$ Distribution", fontsize = "x-large", y = .65)

ax1.plot(x_chi1_abu_exg, h_chi1_abu_exg, label = "mean")
ax2.plot(x_chi1_mfe_exg, h_chi1_mfe_exg, label = "mean")
ax3.plot(x_chi1_dfe_exg, h_chi1_dfe_exg, label = "mean")
ax4.plot(x_chi1_tfe_exg, h_chi1_tfe_exg, label = "mean")

ax1.errorbar(x_chi1_abu_exg, h_chi1_abu_exg, yerr = error_chi1_abu_exg, label = "error", alpha = 0.5)
ax2.errorbar(x_chi1_mfe_exg, h_chi1_mfe_exg, yerr = error_chi1_mfe_exg, label = "error", alpha = 0.5)
ax3.errorbar(x_chi1_dfe_exg, h_chi1_dfe_exg, yerr = error_chi1_dfe_exg, label = "error", alpha = 0.5)
ax4.errorbar(x_chi1_tfe_exg, h_chi1_tfe_exg, yerr = error_chi1_tfe_exg, label = "error", alpha = 0.5)

ax1.legend()
ax2.legend()
ax3.legend()
ax4.legend()

ax1.set_title("Abu")
ax2.set_title("MfeGLy")
ax3.set_title("DfeGly")
ax4.set_title("TfeGly")

ax1.set_box_aspect(1)
ax2.set_box_aspect(1)
ax3.set_box_aspect(1)
ax4.set_box_aspect(1)


fig.text(0.5, 0.32, "$\chi_1$ Angle", ha='center', fontsize = 12)
fig.text(-0.02, 0.48, 'Probability Density', va='center', rotation='vertical', fontsize = 12)

plt.ylim(0, 0.028)
plt.xlim(-180,180)
plt.xticks([-120, -60, 0, 60, 120])
plt.tight_layout()
plt.show()
#fig.savefig("graph/500_chi1_mean_error.png", bbox_inches='tight')

**INTERPRETATION**

Plotting the distribution of the side chain torsion angle $\chi_1$ can also be used to visualize the flexibility of the hexapeptides. Plotted as histogram and weighing the probability density, the side chain torsion angle $\chi_1$ profiles reveal that there is an increasing trend of the trans conformation ($+/-$180°), which happens simultaneously with the decrease of the gauche(+) conformation (-60°). This can be interpreted that the more substituted a hexapeptide with fluorine is, the higher is the tendency to be in a trans conformation due to the steric hinderance with other atoms in the molecule. The gauche(-) conformation (+60°), on the other hand, seems to be stagnant as the more fluorinated a hexapeptide is, and there is no meaningful difference in the result.

We can also see that the errors are relatively large—this is also an indication that the sample size (length of the trajectory) is too small, hence we need a longer simulation.

# end-to-end distance

begin: N-capping @ 0

end: C-capping @ 67

**SLICED!**

In [ ]:
trajectory_abu_500.topology.to_dataframe()

In [ ]:
md.compute_distances(abu_1, [[0,67]], periodic=True)

In [ ]:
for i in name:
    for j in range(1,6):
        globals()['cd_%s_%s' % (i,j)] = md.compute_distances(globals()['%s_%s' % (i,j)], [[0,67]], periodic=True)

In [ ]:
cd_abu = md.compute_distances(traj_abu, [[0,67]], periodic=True)
cd_mfe = md.compute_distances(traj_mfe, [[0,67]], periodic=True)
cd_dfe = md.compute_distances(traj_dfe, [[0,67]], periodic=True)
cd_tfe = md.compute_distances(traj_tfe, [[0,67]], periodic=True)


In [ ]:
cd_abu

In [ ]:
plt.hist(cd_tfe)

In [ ]:
cd_abu.shape

In [ ]:
fig, ax = plt.subplots(1,4, figsize=(12,3), sharex=True, sharey=True)

plt.suptitle("End-to-End distance of the Total 100 ns Trajectory")

sns.histplot(cd_abu, stat='density', bins=100, kde=True, linewidth=0, ax=ax[0])
sns.histplot(cd_mfe, stat='density', bins=100, kde=True, linewidth=0, ax=ax[1])
sns.histplot(cd_dfe, stat='density', bins=100, kde=True, linewidth=0, ax=ax[2])
sns.histplot(cd_tfe, stat='density', bins=100, kde=True, linewidth=0, ax=ax[3])

fig.text(0.4,0, "Distance [nm]")

plt.show()

## distribution

In [ ]:
#h_abu, xedges_abu = np.histogram(backbone_rmsd_abu_500, bins=60, density=True,)

#mids = (xedges_abu[1:] + xedges_abu[:-1]) / 2
#mids_mfe = (xedges_mfe[1:] + xedges_mfe[:-1]) / 2
#mids_dfe = (xedges_dfe[1:] + xedges_dfe[:-1]) / 2
#mids_tfe = (xedges_tfe[1:] + xedges_tfe[:-1]) / 2


In [ ]:
n = 100

for i in name:
    for j in range(1,6):
        globals()['h_ee_%s_%s' % (i,j)], globals()['e_ee_%s_%s' % (i,j)] = np.histogram(
            globals()['cd_%s_%s' % (i,j)],
            bins = n,
            density = True)
        
for i in name:
    for j in range(1,6):
        globals()['x_%s' % i] = (globals()['e_ee_%s_%s' % (i,j)][1:] + globals()['e_ee_%s_%s' % (i,j)][:-1]) /2

In [ ]:
x_tfe

In [ ]:
min(cd_abu)

In [ ]:
min(x_abu)

In [ ]:
#test_array_histogram_1 = np.array([h_ee_dfe_1, h_ee_dfe_2])
#np.mean(test_array_histogram_1, axis=0)

array_abu = np.array([h_ee_abu_1, h_ee_abu_2, h_ee_abu_3, h_ee_abu_4, h_ee_abu_5])
array_mfe = np.array([h_ee_mfe_1, h_ee_mfe_2, h_ee_mfe_3, h_ee_mfe_4, h_ee_mfe_5])
array_dfe = np.array([h_ee_dfe_1, h_ee_dfe_2, h_ee_dfe_3, h_ee_dfe_4, h_ee_dfe_5])
array_tfe = np.array([h_ee_tfe_1, h_ee_tfe_2, h_ee_tfe_3, h_ee_tfe_4, h_ee_tfe_5])

mean_abu = np.mean(array_abu, axis = 0)
mean_mfe = np.mean(array_mfe, axis = 0)
mean_dfe = np.mean(array_dfe, axis = 0)
mean_tfe = np.mean(array_tfe, axis = 0)

std_abu = np.std(array_abu, axis = 0)
std_mfe = np.std(array_mfe, axis = 0)
std_dfe = np.std(array_dfe, axis = 0)
std_tfe = np.std(array_tfe, axis = 0)

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, sharex = True, sharey = True, figsize = (10,10))
fig.suptitle("Averaged End-to-End Distance", fontsize = "x-large", y = .65)


ax1.plot(x_abu, mean_abu, label = "mean")
ax2.plot(x_mfe, mean_mfe, label = "mean")
ax3.plot(x_dfe, mean_dfe, label = "mean")
ax4.plot(x_tfe, mean_tfe, label = "mean")

ax1.set_title("Abu")
ax2.set_title("MfeGLy")
ax3.set_title("DfeGly")
ax4.set_title("TfeGly")

ax1.set_box_aspect(1)
ax2.set_box_aspect(1)
ax3.set_box_aspect(1)
ax4.set_box_aspect(1)

ax1.errorbar(x_abu, mean_abu, yerr = std_abu, label = "error", alpha=0.5)
ax2.errorbar(x_mfe, mean_mfe, yerr = std_mfe, label = "error", alpha=0.5)
ax3.errorbar(x_dfe, mean_dfe, yerr = std_dfe, label = "error", alpha=0.5)
ax4.errorbar(x_tfe, mean_tfe, yerr = std_tfe, label = "error", alpha=0.5)

ax1.legend()
ax2.legend()
ax3.legend()
ax4.legend()

fig.text(0.5, 0.37, "distance [nm]", ha = "center", fontsize = 12)
fig.text(0.05, 0.5, "Probability", va = "center", rotation ="vertical", fontsize = 12)
plt.xticks([0.5, 1, 1.5, 2])
plt.ylim(0)
    
plt.show()

**INTERPRETATION**

The end-to-end distance of each peptide was measured from the C-Atom at the glycine C-terminus to the N-Atom at amide of the valine N-Terminus. The results were plotted as histograms, weighing their probability density. The two peaks in the plot can be interpreted as $\alpha$ helix or closed state and $\beta$ sheet or open states. Generally, as the more substituted a hexapeptide is, the higher their probability density in the closed system is. Yet if it's being observed closer, there is a slight shift in their distance (tab. \ref{fig:tabe2e}) with decreasing distance from Abu to TfeGly. Probably this can be interpreted that the higher fluorine-subsituted hexapeptide do have stronger hydrogen bonds, so that the hexapeptides in both as $\alpha$ helices and $\beta$ sheets structure tend to bond tighter to itself.



# hydrogen bonds

In [ ]:
hbond_abu_1 = md.wernet_nilsson(abu_1, exclude_water=True, periodic=False)
hbond_abu_2 = md.wernet_nilsson(abu_2, exclude_water=True, periodic=False)
hbond_abu_3 = md.wernet_nilsson(abu_3, exclude_water=True, periodic=False)
hbond_abu_4 = md.wernet_nilsson(abu_4, exclude_water=True, periodic=False)
hbond_abu_5 = md.wernet_nilsson(abu_5, exclude_water=True, periodic=False)

hbond_mfe_1 = md.wernet_nilsson(mfe_1, exclude_water=True, periodic=False)
hbond_mfe_2 = md.wernet_nilsson(mfe_2, exclude_water=True, periodic=False)
hbond_mfe_3 = md.wernet_nilsson(mfe_3, exclude_water=True, periodic=False)
hbond_mfe_4 = md.wernet_nilsson(mfe_4, exclude_water=True, periodic=False)
hbond_mfe_5 = md.wernet_nilsson(mfe_5, exclude_water=True, periodic=False)

hbond_dfe_1 = md.wernet_nilsson(dfe_1, exclude_water=True, periodic=False)
hbond_dfe_2 = md.wernet_nilsson(dfe_2, exclude_water=True, periodic=False)
hbond_dfe_3 = md.wernet_nilsson(dfe_3, exclude_water=True, periodic=False)
hbond_dfe_4 = md.wernet_nilsson(dfe_4, exclude_water=True, periodic=False)
hbond_dfe_5 = md.wernet_nilsson(dfe_5, exclude_water=True, periodic=False)

hbond_tfe_1 = md.wernet_nilsson(tfe_1, exclude_water=True, periodic=False)
hbond_tfe_2 = md.wernet_nilsson(tfe_2, exclude_water=True, periodic=False)
hbond_tfe_3 = md.wernet_nilsson(tfe_3, exclude_water=True, periodic=False)
hbond_tfe_4 = md.wernet_nilsson(tfe_4, exclude_water=True, periodic=False)
hbond_tfe_5 = md.wernet_nilsson(tfe_5, exclude_water=True, periodic=False)

In [ ]:
hbond_abu = md.wernet_nilsson(traj_abu, exclude_water=True, periodic=False)
hbond_mfe = md.wernet_nilsson(traj_mfe, exclude_water=True, periodic=False)
hbond_dfe = md.wernet_nilsson(traj_dfe, exclude_water=True, periodic=False)
hbond_tfe = md.wernet_nilsson(traj_tfe, exclude_water=True, periodic=False)


In [ ]:
n = 1

def hbond(x):
    "calculating hydrogen bonds in abu, in all timesteps"
    return md.wernet_nilsson(x, exclude_water = True, periodic = False)


In [ ]:
#def itn_abu(trajectory_abu_100, hbond_abu):
#    """Convert hydrogen bond index tuple to name"""
#    return f"{trajectory_abu_100.topology.atom(hbond_abu[0])}–{trajectory_abu_100.topology.atom(hbond_abu[2])}"

In [ ]:
def hbond_name(traj, hbond_index):
    """Convert hydrogen bond index tuple to name"""
    return f"{traj.topology.atom(hbond_index[0])}–{traj.topology.atom(hbond_index[2])}"

In [ ]:
from collections import defaultdict

In [ ]:
# Count bonds
hb_frequency = defaultdict(int)

In [ ]:
for frame in hbond_abu_1:
    for bond in frame:
        hb_frequency[tuple(bond)] += 1

In [ ]:
def hbond_count(traj):
    for k, v in hb_frequency.items():
        print(f"{hbond_name(traj, k)}: {v}")
    print(f"{hbond_name(traj, k)}: {v}")
    print(k)
    print('')

In [ ]:
hbond_count(traj_abu)

In [ ]:
hbond_count(traj_mfe) == hbond_count(traj_dfe) == hbond_count(traj_tfe)

In [ ]:
keys = list(map(lambda keys: hbond_name(tfe_1, keys), hb_frequency.keys()))

In [ ]:
print(keys)

In [ ]:
def mer(dict1, dict2):
    return(dict1.update(dict2))

In [ ]:
temp = {}
temp1 = {}

In [ ]:
mer(temp, hb_frequency)

In [ ]:
def h_perc(h_dict):
    total = len(tfe_3)
    for j in h_dict:
        h_dict[j] = (float)(h_dict[j])/total*100
    return h_dict

In [ ]:
h_perc(temp)

In [ ]:
#rounding up the decimals
k = 5

#loop to iterate values
res = dict()
for key in temp:
    res[key] = round(temp[key], k)
    
print("rounded up:" + str(res) )

In [ ]:
#rounding up the decimals
k = 1

#loop to iterate values
percentage = dict()
for key in temp1:
    percentage[key] = round(temp1[key], k)
    
print("rounded up:" + str(percentage) )

In [ ]:
fig, ax = plt.subplots()
fig.suptitle("Frequency of Hydrogen Bonds in VG(TfeGly)APG", fontsize = "x-large")

ax.barh(keys, res.values())
for i, v in enumerate(res.values()):
    ax.text(v + 0.1, i - 0.1, str(v))



plt.xlim(0, 6)
plt.viridis()
plt.xlabel("frequency (in percent)")
plt.ylabel("h-bonds")
plt.show()


Although previous papers stated that an amino acid with at least one fluorine substitution will be more hydrophobic and waters are more tightly held at DfeGly and TfeGly\cite{2015} , this thesis proves otherwise. The non-fluorinated hexapeptide itself is hydrophobic \cite{2017} , but there is hardly any difference in the intramolecular hydrogen bonds between the any of the hexapeptides.